# Mini-project: Review of FTSE stocks since Brexit, part 1 

## Objective

The objective is to review all companies in the FTSE 100 and FTSE 250 indices for the two year period following the exit of Britain from the European Union, to see what can be learnt.

**Part 1** will take a list of FTSE stocks, collect the stock price at the start and end of the period, and identifiy the best performers for further analyis.

**Part 2** will look at the trends against other financial metrics and review the data using graphical display tools.


## Skills demonstrated

In completing this section, I have had to:

### Import data

- Imported a list of companies to be reviewed from a csv file created based on information found on the London Stock Exchange website.


- Import data from the **Yahoo Finance API** using the stock tickers, via the **pandas_datareader** library


- Work with the **datetime** feature to retrieve information

### Use basic Python programming skills

- Created a list of stock tickers using a **loop**, whilst adding the correct ticker suffix based on **conditional logic** (if statement)


- **Work with lists** replacing or removing tickers to correct NaN errors 



### Filter/clean large datasets

The data was returned as two dataframes (one for each date), each showing ~350 stocks along with opening, closing, maximum and minimum price, along with volume. 

- The initial dataframes were sorted to only show the adjusted closing price on each date.


- The dataframes were **transposed** to show the company list as rows rather than column headings


- The resulting dataframes were merged, and the columns renamed based on the date


- New columns were created calculating the price increase/decrease in absolute and percentage terms


- A final **filtered dataframe** was created, and merged with the list from the London Stock Exchange to show the company name.


- The final result, which took a long time to run, was then exported to csv format for safekeeping. 

### Import necessary packages

In [1]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr 

### Import iist of FTSE companies from csv file created from information on London Stock Exchange website

In [2]:
ftse_list = pd.read_csv('FTSE.csv')
ftse_list.head()

,Ticker,Name,Market,Unnamed: 3
0,888,888 HOLDINGS PLC ORD 0.5P (DI),FTSE 250,NaN
1,3IN,3I INFRASTRUCTURE PLC ORD NPV,FTSE 250,NaN
2,AAF,AIRTEL AFRICA PLC ORD USD0.50,FTSE 100,NaN
3,AAL,ANGLO AMERICAN PLC ORD USD0.54945,FTSE 100,NaN
4,ABDN,ABRDN PLC ORD 13 61/63P,FTSE 250,NaN


### Drop unneeded column and check result

In [3]:
ftse_list.drop(columns=['Unnamed: 3'], inplace=True)

In [4]:
ftse_list.head()

,Ticker,Name,Market
0,888,888 HOLDINGS PLC ORD 0.5P (DI),FTSE 250
1,3IN,3I INFRASTRUCTURE PLC ORD NPV,FTSE 250
2,AAF,AIRTEL AFRICA PLC ORD USD0.50,FTSE 100
3,AAL,ANGLO AMERICAN PLC ORD USD0.54945,FTSE 100
4,ABDN,ABRDN PLC ORD 13 61/63P,FTSE 250


### Create a list of stock tickers to be passed to the Yahoo Finance API

To differentiate the stocks held on different exchanges, the suffix ".L" needs to be added when working with the Yahoo Finance API.

However, since some tickers with a two digit ticker already contain a full stop (AP, BA, BP etc), we cannot just create a new column adding +'.L' to all the symbols from the LSE.

Here we create a new list of all the yahoo tickers, including the appropriate suffix based on the original format:

In [5]:
ticker_list = []

for i in ftse_list['Ticker']:
    if i[-1] == ".":
        ticker_list.append(i + "L")  
    else:
        ticker_list.append(i + ".L")  

print(ticker_list)

['888.L', '3IN.L', 'AAF.L', 'AAL.L', 'ABDN.L', 'ABF.L', 'ADM.L', 'AGR.L', 'AGT.L', 'AHT.L', 'AJB.L', 'AML.L', 'ANTO.L', 'APAX.L', 'APEO.L', 'ASC.L', 'ASCL.L', 'ASHM.L', 'ASL.L', 'ATG.L', 'ATST.L', 'ATT.L', 'AUTO.L', 'AV.L', 'AVV.L', 'AZN.L', 'BA.L', 'BAB.L', 'BARC.L', 'BATS.L', 'BBGI.L', 'BBH.L', 'BBOX.L', 'BBY.L', 'BCG.L', 'BCPT.L', 'BDEV.L', 'BEZ.L', 'BGEO.L', 'BGFD.L', 'BHMG.L', 'BIFF.L', 'BKG.L', 'BLND.L', 'BME.L', 'BNKR.L', 'BNZL.L', 'BOY.L', 'BP.L', 'BPT.L', 'BRBY.L', 'BRSC.L', 'BRWM.L', 'BSIF.L', 'BT.A.L', 'BVIC.L', 'BWY.L', 'BYG.L', 'BYIT.L', 'CAPC.L', 'CBG.L', 'CCC.L', 'CCH.L', 'CCL.L', 'CCR.L', 'CEY.L', 'CGT.L', 'CHG.L', 'CKN.L', 'CLDN.L', 'CLI.L', 'CMCX.L', 'CNA.L', 'CNE.L', 'COA.L', 'CPG.L', 'CRDA.L', 'CRH.L', 'CRST.L', 'CSP.L', 'CTEC.L', 'CTY.L', 'CURY.L', 'CWK.L', 'DARK.L', 'DCC.L', 'DEC.L', 'DGE.L', 'DLG.L', 'DLN.L', 'DNLM.L', 'DOCS.L', 'DOM.L', 'DPH.L', 'DPLM.L', 'DRX.L', 'DSCV.L', 'EBOX.L', 'EDIN.L', 'EDV.L', 'ELM.L', 'EMG.L', 'ENOG.L', 'ENT.L', 'EOT.L', 'ERM.L', 'ESNT

### Resolve errors:

The following symbols generated an error in the retrieval of information from the Yahoo API.

'GLEN .L' included an extra space and needs to be replaced with the correct ticker: 

In [6]:
ticker_list.remove('GLEN   .L')
ticker_list.append('GLEN.L')

The items listed below returning NaN numbers and will be removed for the purposes of this mini project (some were new stocks, others were errors that in reality each one would be investigated more thoroughly):

    'ATG.L'
    'ATT.L',
    'BCG.L',
    'BPT.L',
    'BT.A.L',
    'BYIT.L',
    'DARK.L',
    'DOCS.L',
    'HBR.L',
    'HLN.L',
    'HOME.L',
    'MOON.L',
    'N91.L',
    'PHLL.L',
    'WPS.L'
    
These will be added to a nan_list, each of which will then be removed from the ticker_list above.

In [7]:
nan_list = ['ATG.L', 'ATST.L', 'ATT.L', 'BCG.L', 'BCPT.L', 'BPT.L', 'BT.A.L', 'BYIT.L', 'DARK.L', 'DOCS.L', 'EBOX.L', 'HBR.L', 'HLN.L', 'HOME.L', 'MOON.L', 'N91.L', 'PHLL.L', 'WPS.L']

In [8]:
for i in nan_list:
    ticker_list.remove(i)

In [9]:
print(ticker_list)

['888.L', '3IN.L', 'AAF.L', 'AAL.L', 'ABDN.L', 'ABF.L', 'ADM.L', 'AGR.L', 'AGT.L', 'AHT.L', 'AJB.L', 'AML.L', 'ANTO.L', 'APAX.L', 'APEO.L', 'ASC.L', 'ASCL.L', 'ASHM.L', 'ASL.L', 'AUTO.L', 'AV.L', 'AVV.L', 'AZN.L', 'BA.L', 'BAB.L', 'BARC.L', 'BATS.L', 'BBGI.L', 'BBH.L', 'BBOX.L', 'BBY.L', 'BDEV.L', 'BEZ.L', 'BGEO.L', 'BGFD.L', 'BHMG.L', 'BIFF.L', 'BKG.L', 'BLND.L', 'BME.L', 'BNKR.L', 'BNZL.L', 'BOY.L', 'BP.L', 'BRBY.L', 'BRSC.L', 'BRWM.L', 'BSIF.L', 'BVIC.L', 'BWY.L', 'BYG.L', 'CAPC.L', 'CBG.L', 'CCC.L', 'CCH.L', 'CCL.L', 'CCR.L', 'CEY.L', 'CGT.L', 'CHG.L', 'CKN.L', 'CLDN.L', 'CLI.L', 'CMCX.L', 'CNA.L', 'CNE.L', 'COA.L', 'CPG.L', 'CRDA.L', 'CRH.L', 'CRST.L', 'CSP.L', 'CTEC.L', 'CTY.L', 'CURY.L', 'CWK.L', 'DCC.L', 'DEC.L', 'DGE.L', 'DLG.L', 'DLN.L', 'DNLM.L', 'DOM.L', 'DPH.L', 'DPLM.L', 'DRX.L', 'DSCV.L', 'EDIN.L', 'EDV.L', 'ELM.L', 'EMG.L', 'ENOG.L', 'ENT.L', 'EOT.L', 'ERM.L', 'ESNT.L', 'EWI.L', 'EXPN.L', 'EZJ.L', 'FAN.L', 'FCIT.L', 'FCSS.L', 'FDM.L', 'FEML.L', 'FEV.L', 'FGP.L', 'FGT.L'

## Set the relevant dates

In [10]:
brexit_date = dt.datetime(2020,1,31)
b_plus_2 = dt.datetime(2022,1,31)

## Make the API call for the initial price on Brexit 

In [11]:
yahoo_df = pdr.get_data_yahoo(ticker_list, brexit_date, brexit_date)

In [12]:
yahoo_df.head()

Attributes   Adj Close                                                 \
Symbols          888.L      3IN.L      AAF.L        AAL.L      ABDN.L   
Date                                                                    
2020-01-31  128.365021  286.67923  64.843224  1753.000122  239.246994   

Attributes                                                                ...  \
Symbols           ABF.L        ADM.L      AGR.L       AGT.L        AHT.L  ...   
Date                                                                      ...   
2020-01-31  2578.928467  1995.884399  69.606049  146.019745  2372.657471  ...   

Attributes   Volume                                                           \
Symbols        WG.L   WHR.L  WIZZ.L   WKP.L WOSG.L    WPP.L  WTAN.L    WTB.L   
Date                                                                           
2020-01-31  3574474  169420  230514  181638  61584  5757466  528794  1721778   

Attributes                   
Symbols     WWH.L    GLEN.L  
Date                         
2020-01-31  65475  29959116  

[1 rows x 1992 columns]

## Transpose the resulting dataframe and check the result

In [13]:
comp_df = yahoo_df['Adj Close'].transpose()

In [14]:
comp_df.head()

Date,2020-01-31
Symbols,
888.L,128.365021
3IN.L,286.679230
AAF.L,64.843224
AAL.L,1753.000122
ABDN.L,239.246994


## Make the second API call to get the post Brexit price

In [15]:
postbx = pdr.get_data_yahoo(ticker_list, b_plus_2, b_plus_2)

In [16]:
postbx.head()

Attributes Adj Close                                                    \
Symbols        888.L  3IN.L  AAF.L   AAL.L      ABDN.L   ABF.L   ADM.L   
Date                                                                     
2022-01-31     255.0  346.0  152.5  3228.0  240.899994  1941.0  3144.0   

Attributes                            ...   Volume                          \
Symbols         AGR.L  AGT.L   AHT.L  ...     WG.L   WHR.L  WIZZ.L   WKP.L   
Date                                  ...                                    
2022-01-31  66.900002  204.0  5246.0  ...  3336732  740052  637380  368470   

Attributes                                                      
Symbols     WOSG.L    WPP.L  WTAN.L    WTB.L   WWH.L    GLEN.L  
Date                                                            
2022-01-31  465349  3986889  824394  1590533  165191  38764882  

[1 rows x 1992 columns]

## Tranpose the second table

In [17]:
postbx = postbx['Adj Close'].transpose()

In [18]:
comp_df = comp_df.merge(postbx, on='Symbols', how='left')

In [19]:
comp_df.head()

Date,2020-01-31 00:00:00,2022-01-31 00:00:00
Symbols,,
888.L,128.365021,255.000000
3IN.L,286.679230,346.000000
AAF.L,64.843224,152.500000
AAL.L,1753.000122,3228.000000
ABDN.L,239.246994,240.899994


## Join the manipulated price dataframes received from the API call

In [20]:
comp_df.columns = ['Brexit', '2_years_later']

In [21]:
comp_df.head()

,Brexit,2_years_later
Symbols,,
888.L,128.365021,255.000000
3IN.L,286.679230,346.000000
AAF.L,64.843224,152.500000
AAL.L,1753.000122,3228.000000
ABDN.L,239.246994,240.899994


## Add columns for gain and loss calculations

In [22]:
comp_df['Gain/(loss)'] = comp_df['2_years_later'] - comp_df['Brexit']

In [23]:
comp_df.head()

,Brexit,2_years_later,Gain/(loss)
Symbols,,,
888.L,128.365021,255.000000,126.634979
3IN.L,286.679230,346.000000,59.320770
AAF.L,64.843224,152.500000,87.656776
AAL.L,1753.000122,3228.000000,1474.999878
ABDN.L,239.246994,240.899994,1.653000


In [24]:
comp_df['% Gain/(loss)'] = comp_df['Gain/(loss)'] / comp_df['Brexit'] * 100

In [25]:
comp_df.head()

,Brexit,2_years_later,Gain/(loss),% Gain/(loss)
Symbols,,,,
888.L,128.365021,255.000000,126.634979,98.652248
3IN.L,286.679230,346.000000,59.320770,20.692385
AAF.L,64.843224,152.500000,87.656776,135.182632
AAL.L,1753.000122,3228.000000,1474.999878,84.141459
ABDN.L,239.246994,240.899994,1.653000,0.690918


## Review key statistics in final dataframe

In [26]:
comp_df.describe()

,Brexit,2_years_later,Gain/(loss),% Gain/(loss)
count,332.000000,332.000000,332.000000,332.000000
mean,1067.193416,1225.140627,157.947211,25.912450
std,1391.808668,1674.438327,555.093722,117.766716
min,17.076342,38.759998,-2014.082520,-70.288875
25%,218.213078,237.474998,-19.919159,-4.045596
50%,522.443115,582.250000,50.446274,14.769226
75%,1341.673431,1507.500000,251.300903,36.381220
max,8707.351562,13255.000000,4547.648438,1971.209251


## Import to .csv file for safekeeping 

In [27]:
comp_df.to_csv('FTSE companies since Brexit.csv')

## Create filter variable and apply this to comparison dataframe

In [28]:
filt = comp_df['% Gain/(loss)'] > 100

In [29]:
over_100pct = comp_df[filt]

## Reset the index on the new dataframe to allow the joining on the Ticker variable

In [46]:
over_100pct = over_100pct.reset_index()

In [47]:
over_100pct['Ticker'] = over_100pct['Symbols'].str[:-2]

## Merge results dataframe with ftse_list dataframe to show stock name, and market (FTSE 100 vs FTSE 250)

In [83]:
# We need to alter the ticker in the original FTSE table to match with the new data frame - it contains unwanted spaces 
ftse_list.loc[ftse_list.Ticker == 'GLEN   ']

,Ticker,Name,Market
130,GLEN,GLENCORE PLC ORD USD0.01,FTSE 100


In [86]:
filt = ftse_list['Ticker'] == 'GLEN   '

In [87]:
ftse_list.loc[filt, 'Ticker'] = 'GLEN'

In [88]:
over_100pct.tail()

,index,Symbols,Brexit,2_years_later,Gain/(loss),% Gain/(loss),Ticker
9,9,INDV.L,189.649994,1125.000000,935.350006,493.198015,INDV
10,10,PFD.L,35.869709,118.000000,82.130291,228.968378,PFD
11,11,PNL.L,17.076342,353.686768,336.610426,1971.209251,PNL
12,12,WOSG.L,373.399994,1272.000000,898.600006,240.653460,WOSG
13,13,GLEN.L,189.506653,383.000000,193.493347,102.103723,GLEN


In [89]:
over_100pct.merge(ftse_list, on='Ticker', how='left')

,index,Symbols,Brexit,2_years_later,Gain/(loss),% Gain/(loss),Ticker,Name,Market
0,0,AAF.L,64.843224,152.500000,87.656776,135.182632,AAF,AIRTEL AFRICA PLC ORD USD0.50,FTSE 100
1,1,AHT.L,2372.657471,5246.000000,2873.342529,121.102290,AHT,ASHTEAD GROUP PLC ORD 10P,FTSE 100
2,2,BME.L,281.070312,566.200012,285.129700,101.444260,BME,B&M EUROPEAN VALUE RETAIL S.A. ORD 10P (DI),FTSE 100
3,3,BRWM.L,302.175354,630.000000,327.824646,108.488214,BRWM,BLACKROCK WORLD MINING TRUST PLC ORD 5P,FTSE 250
4,4,DRX.L,237.726257,601.000000,363.273743,152.811787,DRX,DRAX GROUP PLC ORD 11 16/29P,FTSE 250
5,5,FAN.L,234.652161,500.000000,265.347839,113.081354,FAN,VOLUTION GROUP PLC ORD GBP0.01,FTSE 250
6,6,FUTR.L,1277.812622,3142.000000,1864.187378,145.888947,FUTR,FUTURE PLC ORD 15P,FTSE 250
7,7,FXPO.L,80.056702,241.800003,161.743301,202.035930,FXPO,FERREXPO PLC ORD 10P,FTSE 250
8,8,IDS.L,178.129364,440.399994,262.270630,147.236045,IDS,INTERNATIONAL DISTRIBUTIONS SERVICE ORD 1P,FTSE 250
9,9,INDV.L,189.649994,1125.000000,935.350006,493.198015,INDV,INDIVIOR PLC ORD USD0.50,FTSE 250


In [90]:
over_100pct.to_csv('top_performers.csv')

## Conclusion and next steps

- Despite the fear about the negative impact of Brexit, the mean increase in stock price was 25.3% over two years, and there were many companies that managed to do far better than that 


- In particular, FSTE 250 comapanies were among the top performers


- It is acknowledged that there are a multitud of additional factors at work in the ecomomy over this period, not the least of whcih was Covid.

Part 2 will explore the more detailed trends shown by some of these companies.